# Creación de Redes Eléctricas Simples con Pandapower

## Ejemplos

**Ejemplo 6.8:**

In [ ]:
!pip install pandapower['all']

In [ ]:
import pandapower as pp
net = pp.create_empty_network()  # Red vacía

# Barras (nodos)
bus1 = pp.create_bus(net, vn_kv=20., name="Barra 1")
bus2 = pp.create_bus(net, vn_kv=0.4, name="Barra 2")

# Generación (Slack bus)
pp.create_ext_grid(net, bus=bus1, vm_pu=1.0, name="Red Externa")

# Cargas
pp.create_load(net, bus=bus2, p_mw=0.1, q_mvar=0.05, name="Carga Residencial")

# Líneas
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=0.5,
               std_type="NAYY 4x50 SE", name="Línea 1")

# Transformadores
pp.create_transformer(net, hv_bus=bus1, lv_bus=bus2,
                     std_type="0.63 MVA 20/0.4 kV", name="Trafo 1")

**Ejemplo 6.9:**

In [ ]:
# Creación de barras con diferentes niveles de voltaje
bus_slack = pp.create_bus(net, vn_kv=115, name="Slack Bus", type="b")
bus_pv = pp.create_bus(net, vn_kv=23, name="PV Bus", type="b")
bus_pq = pp.create_bus(net, vn_kv=0.48, name="PQ Bus", type="b")

# Visualizar barras creadas
print(net.bus)

**Ejemplo 6.10:**

In [ ]:
pp.create_ext_grid(net, bus=bus_slack, vm_pu=1.02, name="Subestación Principal",
                  max_p_mw=100, min_p_mw=0, max_q_mvar=50, min_q_mvar=-50)
print(net.ext_grid)

**Ejemplo 6.11:**

In [ ]:
# Diferentes tipos de carga
pp.create_load(net, bus=bus_pq, p_mw=1.5, q_mvar=0.75, name="Residencial",
               const_z_percent=0, const_i_percent=100)  # Corriente constante

pp.create_load(net, bus=bus_pq, p_mw=5.0, q_mvar=2.5, name="Comercial",
               const_z_percent=100, const_i_percent=0)  # Impedancia constante

# Carga con factor de potencia
pp.create_load_from_cosphi(net, bus=bus_pq, sn_mva=3.0, cos_phi=0.9,
                           name="Industrial", mode="underexcited")

**Ejemplo 6.12:**

In [ ]:
# Creación de líneas con tipos estándar
line1 = pp.create_line(net, from_bus=bus_slack, to_bus=bus_pv, length_km=15,
                       std_type="149-AL1/24-ST1A 10.0", name="Línea Tronsmisión")

# Línea con parámetros personalizados
line2 = pp.create_line_from_parameters(
    net, from_bus=bus_pv, to_bus=bus_pq, length_km=0.2, name="Línea Distribución",
    r_ohm_per_km=0.162, x_ohm_per_km=0.083, c_nf_per_km=235, max_i_ka=0.142)

print(net.line)

**Ejemplo 6.13:**

In [ ]:
# Transformador estándar
trafo1 = pp.create_transformer(
    net, hv_bus=bus_slack, lv_bus=bus_pv, name="Trafo Principal",
    std_type="25 MVA 110/20 kV")

# Transformador con tap variable
trafo2 = pp.create_transformer_from_parameters(
    net, hv_bus=bus_pv, lv_bus=bus_pq, name="Trafo Distribución",
    sn_mva=0.63, vn_hv_kv=23, vn_lv_kv=0.48, vkr_percent=4.5,
    vk_percent=12, pfe_kw=1.5, i0_percent=0.3, tap_pos=0,
    tap_neutral=0, tap_min=-2, tap_max=2, tap_step_percent=1.5)

**Ejemplo 6.14:**

In [ ]:
pp.create_gen(net, bus=bus_pv, p_mw=2.0, vm_pu=1.02, name="Generador Diésel",
              min_p_mw=0, max_p_mw=3, min_q_mvar=-1, max_q_mvar=1)

**Ejemplo 6.15:**

In [ ]:
pp.create_sgen(net, bus=bus_pq, p_mw=0.5, q_mvar=0, name="Solar PV",
               type="PV", k=1.2, rx=0.4, current_source=True)

**Ejemplo 6.16:**

In [ ]:
import pandas as pd
# Resumen de la red
print(f"La red contiene:\n"
      f"- {len(net.bus)} barras\n"
      f"- {len(net.line)} líneas\n"
      f"- {len(net.trafo)} transformadores\n"
      f"- {len(net.load)} cargas\n"
      f"- {len(net.ext_grid)} barras slack\n"
      f"- {len(net.gen)} generadores")

# Mostrar todos los elementos
print("\nComponentes de la red:")
for element in net.keys():
    if isinstance(net[element], pd.DataFrame) and not net[element].empty:
        print(f"{element}: {len(net[element])} elementos")

**Ejemplo 6.17:**

In [ ]:
# Diagrama unifilar simple
pp.plotting.simple_plot(net, plot_loads=True, plot_gens=True,
                        trafo_size=1.5, line_width=1.2)

# Diagrama interactivo (requiere plotly)
try:
    import plotly
    pp.plotting.simple_plotly(net, show=True)
except ImportError:
    print("Instala plotly para visualización interactiva: pip install plotly")

**Ejemplo 6.18:**

## Ejemplo Completo: Red de Distribución Rural

```python
import pandapower as pp

def crear_red_rural():
    net = pp.create_empty_network()
    
    # Registrar tipo de transformador personalizado
    pp.create_std_type(net, {
        "sn_mva": 5.0,
        "vn_hv_kv": 34.5,
        "vn_lv_kv": 13.8,
        "vk_percent": 8.0,
        "vkr_percent": 0.5,
        "pfe_kw": 5.0,
        "i0_percent": 0.2,
        "shift_degree": 0
    }, name="5 MVA 34.5/13.8 kV", element="trafo")
    
    # Barras
    bus_slack = pp.create_bus(net, vn_kv=34.5, name="Subestación")
    bus1 = pp.create_bus(net, vn_kv=34.5, name="Alimentador Principal")
    bus2 = pp.create_bus(net, vn_kv=34.5, name="Ramal Norte")
    bus3 = pp.create_bus(net, vn_kv=34.5, name="Ramal Sur")
    bus4 = pp.create_bus(net, vn_kv=13.8, name="Transformador Norte")
    bus5 = pp.create_bus(net, vn_kv=13.8, name="Transformador Sur")
    
    # Slack
    pp.create_ext_grid(net, bus_slack, vm_pu=1.02, name="Red Principal")
    
    # Línea principal (usa un tipo estándar para activar librería)
    pp.create_line(net, bus_slack, bus1, 2.0, std_type="149-AL1/24-ST1A 20.0", name="Línea Principal")

    # Líneas definidas manualmente (estimación típica para 34.5 kV rural)
    pp.create_line_from_parameters(net, bus1, bus2, length_km=5.0,
                                   r_ohm_per_km=0.28, x_ohm_per_km=0.12,
                                   c_nf_per_km=200, max_i_ka=0.24,
                                   name="Ramal Norte", type="ol")
    
    pp.create_line_from_parameters(net, bus1, bus3, length_km=6.0,
                                   r_ohm_per_km=0.28, x_ohm_per_km=0.12,
                                   c_nf_per_km=200, max_i_ka=0.24,
                                   name="Ramal Sur", type="ol")
    
    # Transformadores
    pp.create_transformer(net, bus2, bus4, std_type="5 MVA 34.5/13.8 kV", name="Trafo Norte")
    pp.create_transformer(net, bus3, bus5, std_type="5 MVA 34.5/13.8 kV", name="Trafo Sur")
    
    # Cargas
    pp.create_load(net, bus4, p_mw=1.2, q_mvar=0.6, name="Pueblo Norte")
    pp.create_load(net, bus5, p_mw=1.8, q_mvar=0.9, name="Pueblo Sur")
    
    # Generación distribuida
    pp.create_sgen(net, bus4, p_mw=0.5, q_mvar=0.0, name="Granja Solar Norte")
    pp.create_gen(net, bus5, p_mw=0.3, vm_pu=1.0, name="Generador Emergencia")
    
    return net

# Crear red y ejecutar flujo de potencia
red_rural = crear_red_rural()
pp.runpp(red_rural)

# Visualización unifilar simple
pp.plotting.simple_plot(red_rural)

```

In [ ]:
import pandapower as pp

net = pp.create_empty_network()

# Obtener diccionario de tipos disponibles
std_types_dict = pp.available_std_types(net)

# Listar tipos de línea
print("Tipos de línea disponibles:")
for tipo in std_types_dict.get("line", []):
    print("-", tipo)

# Listar tipos de transformador
print("\n Tipos de transformador disponibles:")
for tipo in std_types_dict.get("trafo", []):
    print("-", tipo)


In [ ]:
import pandapower as pp

def crear_red_rural():
    net = pp.create_empty_network()

    # Registrar tipo de transformador personalizado
    pp.create_std_type(net, {
        "sn_mva": 5.0,
        "vn_hv_kv": 34.5,
        "vn_lv_kv": 13.8,
        "vk_percent": 8.0,
        "vkr_percent": 0.5,
        "pfe_kw": 5.0,
        "i0_percent": 0.2,
        "shift_degree": 0
    }, name="5 MVA 34.5/13.8 kV", element="trafo")

    # Barras
    bus_slack = pp.create_bus(net, vn_kv=34.5, name="Subestación")
    bus1 = pp.create_bus(net, vn_kv=34.5, name="Alimentador Principal")
    bus2 = pp.create_bus(net, vn_kv=34.5, name="Ramal Norte")
    bus3 = pp.create_bus(net, vn_kv=34.5, name="Ramal Sur")
    bus4 = pp.create_bus(net, vn_kv=13.8, name="Transformador Norte")
    bus5 = pp.create_bus(net, vn_kv=13.8, name="Transformador Sur")

    # Slack
    pp.create_ext_grid(net, bus_slack, vm_pu=1.02, name="Red Principal")

    # Línea principal (usa un tipo estándar para activar librería)
    pp.create_line(net, bus_slack, bus1, 2.0, std_type="149-AL1/24-ST1A 20.0", name="Línea Principal")

    # Líneas definidas manualmente (estimación típica para 34.5 kV rural)
    pp.create_line_from_parameters(net, bus1, bus2, length_km=5.0,
                                   r_ohm_per_km=0.28, x_ohm_per_km=0.12,
                                   c_nf_per_km=200, max_i_ka=0.24,
                                   name="Ramal Norte", type="ol")

    pp.create_line_from_parameters(net, bus1, bus3, length_km=6.0,
                                   r_ohm_per_km=0.28, x_ohm_per_km=0.12,
                                   c_nf_per_km=200, max_i_ka=0.24,
                                   name="Ramal Sur", type="ol")

    # Transformadores
    pp.create_transformer(net, bus2, bus4, std_type="5 MVA 34.5/13.8 kV", name="Trafo Norte")
    pp.create_transformer(net, bus3, bus5, std_type="5 MVA 34.5/13.8 kV", name="Trafo Sur")

    # Cargas
    pp.create_load(net, bus4, p_mw=1.2, q_mvar=0.6, name="Pueblo Norte")
    pp.create_load(net, bus5, p_mw=1.8, q_mvar=0.9, name="Pueblo Sur")

    # Generación distribuida
    pp.create_sgen(net, bus4, p_mw=0.5, q_mvar=0.0, name="Granja Solar Norte")
    pp.create_gen(net, bus5, p_mw=0.3, vm_pu=1.0, name="Generador Emergencia")

    return net

# Crear red y ejecutar flujo de potencia
red_rural = crear_red_rural()
pp.runpp(red_rural)

# Visualización unifilar simple
pp.plotting.simple_plot(red_rural)


## Ejercicios

**Ejercicio 6.7: Red Industrial**
Crea una red con:
- 1 barra slack (138 kV)
- 1 barra de subtransmisión (34.5 kV)
- 2 barras de distribución (13.8 kV)
- 2 transformadores
- 3 líneas
- 3 cargas industriales (2+ j1 MVA cada una)
- 1 generador de respaldo

**Ejercicio 6.8: Microred**
Modela una microred con:
- 1 barra slack (12.47 kV)
- 3 barras de distribución
- Generación solar
- Generación eólica
- Baterías
- 5 cargas variadas

**Ejercicio 6.9: Análisis de Componentes**
1. Crea una red con 10 componentes diferentes
2. Exporta los parámetros a DataFrames
3. Genera un reporte en Excel con:
   - Configuración de la red
   - Resultados del flujo de potencia
   - Estadísticas clave

## Soluciones a los Ejercicios

### Solución Ejercicio 1: Red Industrial (Buscar y corregir errores)

```python
def crear_red_industrial():
    net = pp.create_empty_network()
    
    # Barras
    bus_slack = pp.create_bus(net, vn_kv=138, name="Slack")
    bus_sub = pp.create_bus(net, vn_kv=34.5, name="Subtransmisión")
    bus_dist1 = pp.create_bus(net, vn_kv=13.8, name="Distribución 1")
    bus_dist2 = pp.create_bus(net, vn_kv=13.8, name="Distribución 2")
    
    # Slack
    pp.create_ext_grid(net, bus_slack, vm_pu=1.0, name="Red Principal")
    
    # Transformadores
    pp.create_transformer(net, bus_slack, bus_sub, "50 MVA 138/34.5 kV", "Trafo Principal")
    pp.create_transformer(net, bus_sub, bus_dist1, "15 MVA 34.5/13.8 kV", "Trafo Dist 1")
    pp.create_transformer(net, bus_sub, bus_dist2, "15 MVA 34.5/13.8 kV", "Trafo Dist 2")
    
    # Líneas
    pp.create_line(net, bus_sub, bus_dist1, 0.5, "149-AL1/24-ST1A 33.0", "Línea Dist 1")
    pp.create_line(net, bus_sub, bus_dist2, 0.8, "149-AL1/24-ST1A 33.0", "Línea Dist 2")
    pp.create_line(net, bus_dist1, bus_dist2, 1.2, "149-AL1/24-ST1A 33.0", "Línea Interconexión")
    
    # Cargas
    pp.create_load(net, bus_dist1, 2.0, 1.0, "Planta 1")
    pp.create_load(net, bus_dist1, 2.5, 1.2, "Planta 2")
    pp.create_load(net, bus_dist2, 3.0, 1.5, "Taller Principal")
    
    # Generador
    pp.create_gen(net, bus_dist2, 4.0, 1.01, "Generador Emergencia",
                  min_p_mw=0, max_p_mw=5, min_q_mvar=-2, max_q_mvar=2)
    
    return net

red_industrial = crear_red_industrial()
pp.runpp(red_industrial)
pp.plotting.simple_plot(red_industrial)
```

### Solución Ejercicio 2: Microred (buscar y corregir errores)

```python
def crear_microred():
    net = pp.create_empty_network()
    
    # Barras
    bus_slack = pp.create_bus(net, vn_kv=12.47, name="Punto de Conexión")
    bus1 = pp.create_bus(net, vn_kv=12.47, name="Alimentador 1")
    bus2 = pp.create_bus(net, vn_kv=0.48, name="CC Solar")
    bus3 = pp.create_bus(net, vn_kv=0.48, name="CC Eólico")
    
    # Slack
    pp.create_ext_grid(net, bus_slack, vm_pu=1.02, name="Red Utility")
    
    # Líneas
    pp.create_line(net, bus_slack, bus1, 0.3, "NAYY 4x150 SE", "Alimentador")
    pp.create_line(net, bus1, bus2, 0.1, "NAYY 4x50 SE", "Derivación Solar")
    pp.create_line(net, bus1, bus3, 0.15, "NAYY 4x50 SE", "Derivación Eólica")
    
    # Transformadores
    pp.create_transformer(net, bus2, bus3, "0.63 MVA 12.47/0.48 kV", "Trafo CC")
    
    # Generación
    pp.create_sgen(net, bus2, 0.5, 0, "Granja Solar", type="PV")
    pp.create_sgen(net, bus3, 0.3, 0.1, "Turbina Eólica", type="WP")
    
    # Baterías (como carga negativa)
    pp.create_load(net, bus3, -0.2, 0, "Sistema Baterías", const_z_percent=0, const_i_percent=100)
    
    # Cargas
    pp.create_load(net, bus1, 0.4, 0.2, "Edificio Administrativo")
    pp.create_load(net, bus2, 0.3, 0.15, "Iluminación Pública")
    pp.create_load(net, bus3, 0.5, 0.25, "Taller Comunitario")
    
    return net

microred = crear_microred()
pp.runpp(microred)
pp.plotting.simple_plot(microred)
```


### Solución Ejercicio 3: Análisis de Componentes (buscar y corregir errores)

```python
# Crear red de ejemplo
net = crear_red_rural()  # Usamos la red rural creada anteriormente
pp.runpp(net)

# 1. Exportar parámetros a DataFrames
with pd.ExcelWriter('reporte_red_electrica.xlsx') as writer:
    # Configuración de la red
    for element in ['bus', 'line', 'trafo', 'load', 'ext_grid', 'gen', 'sgen']:
        net[element].to_excel(writer, sheet_name=element.capitalize())
    
    # Resultados del flujo de potencia
    for res_element in ['res_bus', 'res_line', 'res_trafo', 'res_load', 'res_gen', 'res_sgen']:
        net[res_element].to_excel(writer, sheet_name=res_element)
    
    # Estadísticas clave
    stats = pd.DataFrame({
        'Voltaje Máximo [pu]': [net.res_bus.vm_pu.max()],
        'Voltaje Mínimo [pu]': [net.res_bus.vm_pu.min()],
        'Carga Máxima Líneas [%]': [net.res_line.loading_percent.max()],
        'Pérdidas Totales [MW]': [net.res_line.pl_mw.sum() + net.res_trafo.pl_mw.sum()],
        'Generación Total [MW]': [net.res_gen.p_mw.sum() + net.res_sgen.p_mw.sum()],
        'Demanda Total [MW]': [net.res_load.p_mw.sum()]
    })
    stats.to_excel(writer, sheet_name='Estadisticas', index=False)

print("Reporte generado en 'reporte_red_electrica.xlsx'")
```
